In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import unicodedata

In [2]:
datetime.now()

datetime.datetime(2020, 3, 16, 11, 21, 2, 835769)

In [3]:
url = 'https://www.health.gov.au/news/health-alerts/novel-coronavirus-2019-ncov-health-alert/coronavirus-covid-19-current-situation-and-case-numbers'
content = requests.get(url).content

In [4]:
soup = BeautifulSoup(content)
data = soup.findAll(text=re.compile('As .+')) + \
        soup.findAll(text=re.compile('.*(?:(Australian Capital Territory)|(Queensland)|(Victoria)|(New South Wales)|(South Australia)|(Western Australia)|(Northern Territory)|(Tasmania)|(Diamond Princess))')) 
data = ' '.join(data)
data

'As at 1.00pm on 16\xa0March 2020, there have been  Australian Capital Territory New South Wales Queensland South Australia Tasmania Victoria Western Australia 10 cases, including 1 death, to the Diamond Princess cruise ship repatriation flight from Japan 9 cases\xa0to an aged care facility in New South Wales'

In [5]:
cases_str = unicodedata.normalize("NFKD", data)

In [6]:
cases_str

'As at 1.00pm on 16 March 2020, there have been  Australian Capital Territory New South Wales Queensland South Australia Tasmania Victoria Western Australia 10 cases, including 1 death, to the Diamond Princess cruise ship repatriation flight from Japan 9 cases to an aged care facility in New South Wales'

In [7]:
date_time_str = re.search('As .+ (\d+ \w+ 202\d)' ,cases_str).group(1)
date_time_str

'16 March 2020'

In [8]:
data_date_time = datetime.strptime(date_time_str, '%d %B %Y').strftime('%Y-%m-%d')
data_date_time

'2020-03-16'

In [9]:
# m = re.search('there were (\d+) confirmed cases', cases_str)
# australia = int(m.group(1))
# australia

In [10]:
# m = re.search('(\d+) in New South Wales', cases_str)
# nsw = int(m.group(1))


In [11]:
# m = re.search('(\d+) in Victoria', cases_str)
# victoria = int(m.group(1))

In [12]:
# m = re.search('(\d+) in South Australia', cases_str)
# sa = int(m.group(1))

In [13]:
# m = re.search('(\d+) in Queensland', cases_str)
# qld = int(m.group(1))

In [14]:
# m = re.search('(\d+) in Tasmania', cases_str)
# tas = int(m.group(1))


In [15]:
# m = re.search('(\d+) in .* Australian Capital Territory', cases_str)
# act = int(m.group(1))



In [16]:
# m = re.search('(\d+) in Northern Territory', cases_str)
# nt = int(m.group(1))


In [17]:
# m = re.search('(\d+) .* Diamond Princess', cases_str)
# wa = int(m.group(1))


### Dataframe

In [18]:
df1 = pd.read_html(url)[0]
df1 = df1.set_index(0).T
df1.insert(0, 'date', data_date_time)
df1 = df1.drop(['Location'], axis=1)
df = df1.rename(columns={
    'Total':'australia',
    'Australian Capital Territory':'act',
    'New South Wales':'nsw',
    'Queensland':'qld',
    'South Australia':'sa',
    'Tasmania':'tas',
    'Victoria':'victoria',
    'Western Australia':'wa',
})

In [19]:
# df = pd.DataFrame({'date': data_date_time, 
#                    'australia': australia, 
#                    'nsw': nsw, 
#                    'victoria': victoria, 
#                    'sa': sa, 
#                    'qld': qld, 
#                    'wa': wa,
#                    'act':act,
# #                   'nt': nt,
#                   'tas': tas }, index=[0])
# df = pd.DataFrame({'date': data_date_time, 'australia': australia}, index=[0])

In [20]:
df

0        date act  nsw qld  sa tas victoria  wa australia
1  2020-03-16   1  134  62  20   6       57  18       298

### Save to file

In [21]:
from pathlib import Path
csv_file = '../data/australia-cases.csv'

if Path(csv_file).exists():
    # read out the old data
    old_df = pd.read_csv(csv_file)
    df = df.append(old_df)
df.drop_duplicates(subset=['date'], inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [22]:
df.sort_values(by=['date'], inplace=True)
df.to_csv(csv_file, index=False)
df

act australia        date  nsw   nt  qld   sa  tas victoria   wa
0   NaN        12  2020-02-04    4  NaN    2    2  NaN        4  NaN
1   NaN        13  2020-02-05    4  NaN    3    2  NaN        4  NaN
2   NaN        14  2020-02-06    4  NaN    4    2  NaN        4  NaN
3   NaN        15  2020-02-07    4  NaN    5    2  NaN        4  NaN
4   NaN        15  2020-02-08    4  NaN    5    2  NaN        4  NaN
5   NaN        15  2020-02-09    4  NaN    5    2  NaN        4  NaN
6   NaN        15  2020-02-10    4  NaN    5    2  NaN        4  NaN
7   NaN        15  2020-02-11    4  NaN    5    2  NaN        4  NaN
8   NaN        15  2020-02-12    4  NaN    5    2  NaN        4  NaN
9   NaN        15  2020-02-13    4  NaN    5    2  NaN        4  NaN
10  NaN        15  2020-02-14    4  NaN    5    2  NaN        4  NaN
11  NaN        15  2020-02-16    4  NaN    5    2  NaN        4  NaN
12  NaN        15  2020-02-17    4  NaN    5    2  NaN        4  NaN
13  NaN        15  2020-02-18    4  NaN    5    2  NaN        4  NaN
14  NaN        15  2020-02-19    4  NaN    5    2  NaN        4  NaN
15  NaN        15  2020-02-20    4  NaN    5    2  NaN        4  NaN
16  NaN        17  2020-02-21    4  NaN    5    2  NaN        4  NaN
17  NaN        22  2020-02-22    4  NaN    5    2  NaN        4  NaN
18  NaN        22  2020-02-23    4  NaN    8    3  NaN        6    1
19  NaN        22  2020-02-24    4  NaN    8    3  NaN        6    1
20  NaN        22  2020-02-25    4  NaN    8    3  NaN        6    1
21  NaN        23  2020-02-26    4  NaN    8    3  NaN        7    1
22  NaN        23  2020-02-27    4  NaN    8    3  NaN        7    1
23  NaN        23  2020-02-28    4  NaN    8    3  NaN        7    1
24  NaN        25  2020-02-29    4  NaN    9    3  NaN        7    2
25  NaN        26  2020-03-01    4  NaN    9    3  NaN        8    2
26  NaN        29  2020-03-02    6  NaN    9    3  NaN        9    2
27  NaN        33  2020-03-03  NaN  NaN  NaN  NaN  NaN      NaN  NaN
28  NaN        41  2020-03-04  NaN  NaN  NaN  NaN  NaN      NaN  NaN
29  NaN        52  2020-03-05   22  1.0    8    4    1        6  NaN
30  NaN        59  2020-03-06   25  NaN   10    6    1        6  NaN
31  NaN        63  2020-03-07   28  NaN   11    6    1        6  NaN
32  NaN        74  2020-03-08   38  NaN   11    6    1        7  NaN
33  NaN        80  2020-03-09   40  NaN   12    6    2        8  NaN
34  NaN       100  2020-03-10   54  NaN   12    6    2       12  NaN
35  NaN       112  2020-03-11   60  NaN   15    6    2       15  NaN
36  NaN       126  2020-03-12   64  NaN   17    8    2       18  NaN
37    1       156  2020-03-13   77  NaN   24   11    3       23  NaN
38    1       197  2020-03-14   91  NaN   35   16    4       36  NaN
39    1       249  2020-03-15  111  NaN   46   19    6       49   17
1     1       298  2020-03-16  134  NaN   62   20    6       57   18